<center>

# Riemannian L-systems: Modeling growing forms in curved spaces

## Christophe Godin<sup>1</sup> and Frédéric Boudon<sup>2,3</sup>
</center>


<center>
<sup>1</sup>Laboratoire Reproduction et Développement des Plantes, Univ Lyon, ENS de Lyon, UCB Lyon1, CNRS,
INRAE, Inria, F-69342 Lyon, France
</center>

<center>
<sup>2</sup>CIRAD, UMR AGAP Institute, F-34398 Montpellier, France
</center>
<center>
<sup>3</sup>UMR AGAP Institute, Univ. Montpellier, CIRAD, INRAE, Institute Agro, Montpellier, France
</center>

<center>
christophe.godin@inria.fr, frederic.boudon@cirad.fr
</center>

## Abstract

In the past 50 years, L-systems have been successfully used to model the development of filamentous and branching forms in biology. Their success is largely due to the fact that they rely on the mathematical notion of discrete rewriting systems, that in essence simply reflects the idea that the evolution of a structure results from the evolution of its individual components. This core property is reminiscent of how biological organisms develop and happens to be critical to model their growth. The formalism of L-systems has been developed to model the growth of forms in Euclidean 1-D, 2-D, or 3-D spaces. These spaces have the property to be flat and show no curvature anywhere. However, the growth of various forms or processes in biology takes place in curved spaces. This is for example the case of vein networks growing within curved leaf blades, of unicellular tubes, such as pollen tubes, growing on curved surfaces to fertilize distant ovules, of teeth patterns growing on folded epithelia of animals, of diffusion of chemical or mechanical signals at the surface of plant or animal tissues, etc. To model these growing forms in curved spaces, we thus developed further the theory of L-systems. In a first step we show that this extension can be carried out by integrating concepts of differential geometry in the notion of turtle geometry. We then illustrate how this extension can be applied to model and program the development of both mathematical and biological forms on curved surfaces embedded in our Euclidean space. We provide various examples applied to plant development. We finally show that this approach can be extended to more abstract spaces, called abstract Riemannian spaces, that are not embedded into any higher space, while being intrinsically curved. We suggest that this abstract extension can be used to provide a new approach for effective modeling of tropism phenomena and illustrate this idea on a few conceptual examples.

In [ ]:
import pgljupyter
from display import display_example, display_examples

## 2. L-systems overview
### 2.4 L-system examples in L-Py

In [ ]:
display_example('Fig01-Listing1', animate=True)

In [ ]:
display_example('Fig01-Listing2', animate=True)

In [ ]:
display_example('Fig01-Listing3', animate=True)

In [ ]:
display_example('Fig01-Listing4', animate=True)

## 3. Moving on parametric surfaces
### 3.4 Turning on curved surfaces and holonomy
### Parallel transport

In [ ]:
display_example('Fig08-Listing5.lpy')

### Closed polygons on a surface

In [ ]:
display_example('Fig09.lpy', animate=False)

### Drawing smooth curve on a curved surface

In [ ]:
display_example('Fig10a-e',animate=False)

In [ ]:
display_example('Fig10f-h',animate=False)

In [ ]:
display_example('Fig10i-k',animate=False)

In [ ]:
display_example('Fig10l-n',animate=False)

## 4. Freely growing forms on curved surfaces
### 4.1 Geodesic trajectories

In [ ]:
display_example('Fig11a',animate=False)

In [ ]:
display_example('Fig11b',animate=False)

In [ ]:
display_example('Fig11c')

### 4.2 Turning and branching
### Fractals.

In [ ]:
display_example('Fig12')

### Branching patterns.

In [ ]:
display_example('Fig13a',animate=False, codedisplay=False)

In [ ]:
display_example('Fig13b',animate=False, codedisplay=False)

In [ ]:
display_example('Fig13c',animate=False, codedisplay=False)

In [ ]:
display_example('Fig13d',animate=False, codedisplay=False)

### 4.3 Applications
### Filamentous growth.

In [ ]:
display_example('Fig14', codedisplay=False)

### Branching system growth.

### Figure 15a: Cabbage leaf model.

photos of a white cabbage leaf (up: top view, below: side view).

<img src="./cabbageleaf.png" width=250 height=200>

In [ ]:
display_example('Fig15', animate=False, codedisplay=False)

### Plant branching system.

In [ ]:
display_example('Fig16', animate=False, codedisplay=False)

## 5 Feedbacks between surface and embedded forms

### 5.1 Information feedback

### Scalar fields

In [ ]:
display_example('Fig17a-c', animate=False, codedisplay=False)

In [ ]:
display_example('Fig17d-f', animate=True, codedisplay=False)

### Vector fields

In [ ]:
display_example('Fig18a', animate=False, codedisplay=False)

In [ ]:
display_example('Fig18b', animate=False, codedisplay=False)

In [ ]:
display_example('Fig18c-d', animate=False, codedisplay=False)

### 5.2 Embedding of forms in dynamic surfaces

### Convected versus floating forms

In [ ]:
display_example('Fig19a', animate=False)

In [ ]:
display_example('Fig19b-c', animate=False, codedisplay=False)

### Feedback of the embedding surface growth on the form

In [ ]:
display_example('Fig20a-b', animate=False, codedisplay=False)

In [ ]:
display_example('Fig20c-d', animate=False, codedisplay=False)

### Figure 21a: Kidney fern model. 

Hymenophyllum nephrophyllum (Kidney fern).

<img src="./kidneyfern.png" height=200 width=300>

In [ ]:
display_example('Fig21', animate=False, codedisplay=False)

In [ ]:
display_example('Fig21', animate=False, codedisplay=False)

## 6 L-systems in abstract Riemannian spaces

### 6.2 Examples in 2-D abstract Riemannian spaces


In [ ]:
display_example('Fig22', animate=False, codedisplay=False)

### Immersion of an intrinsic geometry in an abstract space

In [ ]:
display_example('Fig23', animate=False)

### Towards modeling tropism in plants with abstract Riemannian spaces

In [ ]:
display_example('Fig24', animate=False, codedisplay=False)